In [1]:
import os
import sys
os.environ['PATH']+=':'+sys.path[1].split('lib')[0]+'bin'
sys.path.append('/home/diego/Projects')

import tqdm
import tempfile
import numpy as np
import matplotlib.pyplot as plt

import simtk.openmm as openmm
import simtk.unit as unit
import simtk.openmm.app as app
from openmmtools import states, mcmc, testsystems
from yank.multistate import ReplicaExchangeSampler, MultiStateReporter, ReplicaExchangeAnalyzer

plt.style.use("ggplot")

In [2]:
n_replicas = 6  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.

In [3]:
testsystem = testsystems.AlanineDipeptideImplicit()

In [4]:
temperatures = [T_min + (T_max - T_min) * (np.exp(float(i) / float(n_replicas-1)) - 1.0) / (np.e - 1.0) for i in range(n_replicas)]
print(temperatures)

[Quantity(value=298.0, unit=kelvin), Quantity(value=336.91307692192413, unit=kelvin), Quantity(value=384.44161640286114, unit=kelvin), Quantity(value=442.4931056162021, unit=kelvin), Quantity(value=513.3973546566822, unit=kelvin), Quantity(value=600.0, unit=kelvin)]


In [5]:
thermodynamic_states = [states.ThermodynamicState(system=testsystem.system, temperature=T) for T in temperatures]

In [6]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)

In [7]:
simulation = ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=5)

In [8]:
storage_path = tempfile.NamedTemporaryFile(delete=False).name + '.nc'

In [9]:
reporter = MultiStateReporter(storage_path, checkpoint_interval=1)

In [10]:
simulation.create(thermodynamic_states=thermodynamic_states,
                  sampler_states=states.SamplerState(testsystem.positions), storage=reporter)

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [11]:
simulation.run()

In [12]:
simulation.iteration

5

In [13]:
simulation.extend(n_iterations=5)

In [14]:
simulation.iteration

10

In [ ]:
#del simulation
#simulation = ReplicaExchangeSampler.from_storage(reporter)
#simulation.extend(n_iterations=2)
#simulation.iteration

In [15]:
reporter = MultiStateReporter(storage=storage_path, open_mode='r', checkpoint_interval=1)

In [16]:
sampler_states = reporter.read_sampler_states(iteration=1)

In [17]:
len(sampler_states)

6

In [18]:
sampler_states[-1].positions.shape

(22, 3)

In [19]:
os.remove(storage_path)